## double filter + style

In [10]:
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

layout150=widgets.Layout(width= '150px', height='30px',display='flex' ,justify_content="center")
layout100=widgets.Layout(width= '100px', height='50px')
layout50=widgets.Layout(width= '50px', height='50px')

slider = widgets.IntSlider(description='$x$', value=4)
text = widgets.IntText(disabled=True, description='$x^2$',layout=layout150)

def compute(*ignore):
    text.value = str(slider.value ** 2)

slider.observe(compute,'value')
widgets.VBox([slider, text])

In [20]:
import pandas as pd
import numpy as np
url = "international-visitors-london-raw.csv"
df_london = pd.read_csv(url,encoding = "ISO-8859-1")
df_london.columns = ['year', 'quarter', 'market', 'dur_stay', 'mode', 'purpose', 'area', 
                     'Visits', 'Spend', 'Nights', 'sample']
df_london=df_london.sample(1000)
df_london.info()
#df_london.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 18587 to 39553
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1000 non-null   object 
 1   quarter   1000 non-null   object 
 2   market    1000 non-null   object 
 3   dur_stay  1000 non-null   object 
 4   mode      1000 non-null   object 
 5   purpose   1000 non-null   object 
 6   area      1000 non-null   object 
 7   Visits    1000 non-null   float64
 8   Spend     1000 non-null   float64
 9   Nights    1000 non-null   float64
 10  sample    1000 non-null   int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 93.8+ KB


In [12]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
# add dropdown widget and its attribute
dropdown_year =    widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year),layout=layout150)
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.purpose),layout=layout150)
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1,layout=layout150)
# add output widget for rendering result and refresh each time
output = widgets.Output(layout={'border': '1px solid black'})
plot_output = widgets.Output()


In [13]:
def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color:red'
    else:
        return 'color: black'

def common_filtering(year, purpose, num):
    output.clear_output()
    plot_output.clear_output()

    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]

    with output:
        display(common_filter.style.applymap(lambda x:colour_ge_value(x, num),
                                             subset=['Visits','Spend', 'Nights']))
    with plot_output:
        sns.kdeplot(common_filter['Visits'], shade=True)
        plt.show()

def dropdown_year_eventhandler(change):
    common_filtering(change.new,          dropdown_purpose.value, bounded_num.value)
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new,             bounded_num.value)
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)
    

In [14]:
# add observe
dropdown_year.observe   (dropdown_year_eventhandler,    names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler,names='value')
bounded_num.observe(bounded_num_eventhandler,names='value')

#widgets.VBox([dropdown_year, dropdown_purpose, bounded_num,output,plot_output])

# Dashboard

In [15]:
columns_widgets = widgets.HBox([widgets.Label(value='Year',layout=layout150),
                               widgets.Label(value='Purpose',layout=layout150),
                               widgets.Label(value='Minimum',layout=layout150)])

In [16]:
input_widgets = widgets.HBox([dropdown_year, dropdown_purpose, bounded_num])
tab = widgets.Tab([output,plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')
dashboard = widgets.VBox([columns_widgets,input_widgets, tab])
display(dashboard)